## Setup

In [9]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "test"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [10]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
# print(pages[3].page_content)


100%|██████████| 1/1 [00:00<00:00, 45.25it/s]

Number of chunks =  1


## Create a dataframe of all the chunks

In [11]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(1, 3)


,text,source,chunk_id
0,The students should learn the main concepts of...,data_input/test/test.txt,9fe2bdf3a4144dfa9783e0829ccab403


## Extract Concepts

In [12]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [13]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model="gpt-oss:120b-cloud")
    dfg = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)

    dfg.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg.replace("", np.nan, inplace=True)
dfg.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg['count'] = 4
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.

[
{
"node_1": "imperative programming",
"node_2": "types",
"edge": "understanding types is the first step in imperative programming"
},
{
"node_1": "types",
"node_2": "variables",
"edge": "variables store values of defined types"
},
{
"node_1": "variables",
"node_2": "operators",
"edge": "operators act on variables to produce results"
},
{
"node_1": "operators",
"node_2": "expressions",
"edge": "expressions are built from operators and operands"
},
{
"node_1": "expressions",
"node_2": "evaluation",
"edge": "evaluation determines the value of an expression"
},
{
"node_1": "evaluation",
"node_2": "statements",
"edge": "statements execute evaluated expressions"
},
{
"node_1": "statements",
"node_2": "control structures",
"edge": "control structures direct the flow of statements"
},
{
"node_1": "control structures",
"node_2": "comments",
"edge": "comments annotate code without affecting execution"
},
{
"node_1": "comments",
"node_2": "procedural programming",
"edge": "procedural programmin

## Calculate the NetworkX Graph

In [14]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(51,)

In [15]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )


In [16]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
